In [48]:
%%capture
!pip install folium matplotlib mapclassify

In [49]:
import pandas as pd
import geopandas as gpd
import os
import zipfile

FOLDER_PATH = '/content/drive/MyDrive/DSL SMART CITIES'

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import NILs

In [50]:
nil = gpd.read_file(os.path.join(FOLDER_PATH, 'Processed','nil.geojson'))
nil = nil.to_crs(epsg=4326)
nil

,ID_NIL,NIL,area_sqkm,Residenti,geometry
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ..."
1,2,brera,1.637333,18294,"POLYGON ((9.19165 45.46906, 9.19204 45.46937, ..."
2,3,giardini p.ta venezia,0.249637,43,"POLYGON ((9.20090 45.47654, 9.20000 45.47696, ..."
3,4,guastalla,1.547962,15528,"POLYGON ((9.20700 45.46787, 9.20693 45.46853, ..."
4,5,porta vigentina - porta lodovica,1.135196,13548,"POLYGON ((9.20186 45.45238, 9.20078 45.45301, ..."
...,...,...,...,...,...
83,84,parco nord,1.532331,98,"POLYGON ((9.20040 45.52848, 9.20028 45.52846, ..."
84,85,parco delle abbazie,13.733841,364,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ..."
85,86,parco dei navigli,3.623149,354,"POLYGON ((9.13886 45.42855, 9.13769 45.42947, ..."
86,87,assiano,5.840942,211,"POLYGON ((9.04687 45.46276, 9.04685 45.46278, ..."


# Import Age_groups data

In [51]:
agegroups = pd.read_csv(os.path.join(FOLDER_PATH,'agegroups.csv'),sep=';', encoding='latin-1')
agegroups = agegroups.rename(columns={'NIL (Nuclei di Identità Locale)':'NIL',
                                      'Età (classi funzionali)':'age',
                                      'Residenti':'residents'})

agegroups['ID_NIL'] = agegroups['NIL'].str.extract('.*\((.*)\).*')
agegroups

,age,NIL,residents,ID_NIL
0,0-2 anni,Duomo (1),292,1
1,0-2 anni,Brera (2),362,2
2,0-2 anni,Giardini P.ta Venezia (3),1,3
3,0-2 anni,Guastalla (4),319,4
4,0-2 anni,Porta Vigentina - Porta Lodovica (5),257,5
...,...,...,...,...
1122,85 anni e più,Parco Nord (84),1,84
1123,85 anni e più,Parco delle Abbazie (85),5,85
1124,85 anni e più,Parco dei Navigli (86),2,86
1125,85 anni e più,Assiano (87),8,87


In [52]:
agegroups_pivoted = agegroups.pivot(index='ID_NIL', columns='age', values='residents').reset_index().rename_axis(None, axis=1)
agegroups_pivoted

,ID_NIL,0-2 anni,11-13 anni,14-18 anni,19-24 anni,25-34 anni,3-5 anni,35-44 anni,45-54 anni,55-64 anni,6-10 anni,65-74 anni,75-84 anni,85 anni e più
0,1,292.0,470.0,891.0,1071.0,1985.0,342.0,1942.0,2617.0,2576.0,719.0,1658.0,1436.0,609.0
1,10,377.0,410.0,658.0,1097.0,3047.0,351.0,2841.0,2774.0,2537.0,639.0,1651.0,1398.0,630.0
2,11,506.0,518.0,845.0,1062.0,3322.0,513.0,3782.0,3736.0,3060.0,848.0,1966.0,1684.0,799.0
3,12,510.0,634.0,1068.0,1372.0,3490.0,528.0,3648.0,4119.0,3601.0,963.0,2745.0,2204.0,1215.0
4,13,336.0,419.0,719.0,828.0,2246.0,388.0,2500.0,2541.0,2256.0,634.0,1341.0,1234.0,695.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,85,13.0,17.0,22.0,17.0,42.0,15.0,58.0,66.0,40.0,30.0,23.0,16.0,5.0
83,86,12.0,7.0,13.0,20.0,77.0,17.0,48.0,46.0,56.0,21.0,26.0,9.0,2.0
84,87,5.0,8.0,9.0,7.0,30.0,5.0,26.0,37.0,36.0,9.0,19.0,12.0,8.0
85,88,14.0,30.0,46.0,43.0,96.0,16.0,87.0,144.0,80.0,45.0,40.0,38.0,11.0


In [53]:
agegroups_merged = nil.merge(agegroups_pivoted, on='ID_NIL', how='left').fillna(0)
agegroups_merged

,ID_NIL,NIL,area_sqkm,Residenti,geometry,0-2 anni,11-13 anni,14-18 anni,19-24 anni,25-34 anni,3-5 anni,35-44 anni,45-54 anni,55-64 anni,6-10 anni,65-74 anni,75-84 anni,85 anni e più
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",292.0,470.0,891.0,1071.0,1985.0,342.0,1942.0,2617.0,2576.0,719.0,1658.0,1436.0,609.0
1,2,brera,1.637333,18294,"POLYGON ((9.19165 45.46906, 9.19204 45.46937, ...",362.0,481.0,835.0,1009.0,2066.0,403.0,2270.0,2997.0,2823.0,762.0,1930.0,1641.0,715.0
2,3,giardini p.ta venezia,0.249637,43,"POLYGON ((9.20090 45.47654, 9.20000 45.47696, ...",1.0,2.0,3.0,2.0,7.0,0.0,3.0,7.0,9.0,2.0,4.0,3.0,0.0
3,4,guastalla,1.547962,15528,"POLYGON ((9.20700 45.46787, 9.20693 45.46853, ...",319.0,450.0,719.0,922.0,1744.0,306.0,1817.0,2445.0,2355.0,629.0,1670.0,1462.0,690.0
4,5,porta vigentina - porta lodovica,1.135196,13548,"POLYGON ((9.20186 45.45238, 9.20078 45.45301, ...",257.0,399.0,630.0,934.0,1798.0,261.0,1613.0,2001.0,1987.0,541.0,1289.0,1227.0,611.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,parco nord,1.532331,98,"POLYGON ((9.20040 45.52848, 9.20028 45.52846, ...",3.0,3.0,7.0,5.0,10.0,1.0,13.0,24.0,16.0,3.0,7.0,5.0,1.0
84,85,parco delle abbazie,13.733841,364,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ...",13.0,17.0,22.0,17.0,42.0,15.0,58.0,66.0,40.0,30.0,23.0,16.0,5.0
85,86,parco dei navigli,3.623149,354,"POLYGON ((9.13886 45.42855, 9.13769 45.42947, ...",12.0,7.0,13.0,20.0,77.0,17.0,48.0,46.0,56.0,21.0,26.0,9.0,2.0
86,87,assiano,5.840942,211,"POLYGON ((9.04687 45.46276, 9.04685 45.46278, ...",5.0,8.0,9.0,7.0,30.0,5.0,26.0,37.0,36.0,9.0,19.0,12.0,8.0


In [54]:
agegroups_merged.to_csv(os.path.join(FOLDER_PATH,'Processed','agegroups.csv'), index=False)